# Object-Oriented Programming (Part 2)

* Now that we've looked at decorators, we can delve deeper into object-oriented programming

In [1]:
class Duck:
    def __init__(self, name):
        self.hidden_name = name
        
    def get_name(self):
        """getter for name attribute"""
        print('Inside the getter')
        return self.hidden_name

    def set_name(self, val):
        """setter for name attribute"""
        print('Inside the setter')
        self.hidden_name = val
    
    # the property() class returns a special descriptor object
    name = property(get_name, set_name)

In [2]:
dir(property())

['__class__',
 '__delattr__',
 '__delete__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__isabstractmethod__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__set__',
 '__set_name__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'deleter',
 'fdel',
 'fget',
 'fset',
 'getter',
 'setter']

In [3]:
property().getter

<function property.getter>

In [4]:
property().setter

<function property.setter>

In [5]:
fowl = Duck('Donald')

In [6]:
fowl.name

Inside the getter


'Donald'

In [7]:
fowl.name = 'foo' # invokes the set_name function

Inside the setter


In [8]:
fowl.get_name()

Inside the getter


'foo'

In [9]:
fowl.name = 'Daffy'
fowl.name

Inside the setter
Inside the getter


'Daffy'

In [10]:
class Duck:
    def __init__(self, name):
        self._hidden_name = name
        
    @property
    def name(self): #
        '''getter for name attribute'''
        print('Inside the getter')
        return self._hidden_name
    
    #name = property(name)
    
    @name.setter
    def name(self, val):
        '''setter for name attribute'''
        print('Inside the setter')
        self._hidden_name = val

In [11]:
fowl = Duck('Donald')
fowl.name # we no longer have get_name and set_name functions

Inside the getter


'Donald'

In [12]:
# but hidden_name can still be accessed from outside
fowl.name = 'Marc'
fowl.name, fowl._hidden_name

Inside the setter
Inside the getter


('Marc', 'Marc')

In [13]:
class Duck():
    def __init__(self, name):
        # data which is intended to be truly private can be preceded with "dunder"
        self.__name = name
        
    @property
    def name(self):
        """getter for name attribute"""
        print('in the getter')
        return self.__name
    
    # name = property(name)
    
    @name.setter
    def name(self, val):
        """setter for name attribute"""
        self.__name = val


In [14]:
d = Duck('Donald')
d.name

in the getter


'Donald'

In [15]:
d.__name # finally private?

AttributeError: 'Duck' object has no attribute '__name'

In [16]:
d.__dict__

{'_Duck__name': 'Donald'}

In [17]:
# not quite ... __name is mangled cannot be accessed 
# except by its mangled name
d._Duck__name

'Donald'

# Class and Static Methods
* class methods are methods that operate on the class itself, rather than instance of the class
* static methods are methods that don't operate on an instance of the object nor do they operate on the class

In [18]:
class Duck:
    _species = 'fowl' # class data
    
    def __init__(self, name):
        # data which is intended to be truly private can be preceeded with "dunder"
        self.__name = name
        
    @property
    def name(self):
        '''getter for name attribute'''
        print('in getter')
        return self.__name
    
    @name.setter
    def name(self, val):
        '''setter for name attribute'''
        print('IN SETTER')
        self.__name = val
        
    @staticmethod
    def myprint(thing):
        '''note that self is NOT the first param'''
        print('-' * len(thing), thing, '-' * len(thing), sep='\n')

    #myprint = staticmethod(myprint)

In [19]:
d = Duck('Marc')
Duck.myprint('Taylor Swift')

------------
Taylor Swift
------------


In [20]:
d.name = 'Taylor'

IN SETTER


In [21]:
d.name

in getter


'Taylor'

In [22]:
d.__dict__

{'_Duck__name': 'Taylor'}

In [23]:
d._Duck__name

'Taylor'

In [24]:
class Example:
    # class data, shared by all instances of thre class
    __some_data = 'blah'
    __how_many = 0
    
    def __init__(self, val):
        print('in init for Example')
        self.name = val # instance data
        self.__class__.__how_many += 1 # get from object to class
        print('__how_many =', self.__class__.__how_many)

    def __del__(self):
        self.__class__.__how_many -= 1
        
    # We can use a static (or class) method to get around
    # a brittle __init__ that doesn't quite do what we want.
    @staticmethod
    def list_init(somelist):
        '''allow me to send in a list, and "flatten" it
        into a string with intervening commas'''
        obj = Example('')
        obj.name = ', '.join(somelist)
        return obj
    
    @classmethod
    def get_some_data(cls):
        return cls.__some_data
    
    @classmethod
    def get_count(cls):
        return cls.__how_many

In [25]:
a = Example('foo')

in init for Example
__how_many = 1


In [26]:
b = Example.list_init(['a', 'b', 'c'])
# b = Example('')
# b.name = ...

in init for Example
__how_many = 2


In [27]:
Example.get_count()

2

In [28]:
b.name

'a, b, c'

In [29]:
e = Example('foo')
e2 = Example.list_init(['foo', 'bar', 'baz'])
print(type(e), e.name, e2.name, type(e2), sep='\n')
print(Example.get_count())

in init for Example
__how_many = 3
in init for Example
__how_many = 4
<class '__main__.Example'>
foo
foo, bar, baz
<class '__main__.Example'>
4


In [30]:
del e
print(Example.get_count())

3


In [31]:
e3 = e2

In [32]:
del e3

In [33]:
e2

# Lab: Class Methods
* add class methods to your class which keeps track of all the instance names which have been created
  * __`.allnames()`__ should return a list of all the names of objects which exist
  * __`.count()`__ should return the number of objects that have ever been created
  * we will need `__del__` to accomplish this

In [34]:
class InstanceTracker:
    _all_names = set()  # Stores currently existing instance names
    _count = 0          # Tracks total objects ever created

    def __init__(self, name):
        self.name = name
        InstanceTracker._all_names.add(name)
        InstanceTracker._count += 1  # Increment total count

    def __del__(self):
        """Removes name from active instances when the object is deleted."""
        InstanceTracker._all_names.discard(self.name)

    @classmethod
    def allnames(cls):
        """Returns a list of currently existing instance names."""
        return list(cls._all_names)

    @classmethod
    def count(cls):
        """Returns the number of objects that have ever been created."""
        return cls._count

# Example usage:
a = InstanceTracker("Alice")
b = InstanceTracker("Bob")
c = InstanceTracker("Charlie")

print(InstanceTracker.allnames())  # ['Alice', 'Bob', 'Charlie']
print(InstanceTracker.count())     # 3 (total objects ever created)

del a  # Deleting 'Alice'

print(InstanceTracker.allnames())  # ['Bob', 'Charlie']
print(InstanceTracker.count())     # Still 3 (count never decreases)


['Alice', 'Charlie', 'Bob']
3
['Charlie', 'Bob']
3


## The Python Data Model
* also from Luciano Ramalho...
* let's return to our Pythonic deck of cards
  * we used named tuples to represent each card
  * the 'deck' is simply a list of cards

In [35]:
import collections

Card = collections.namedtuple('Card', 'rank suit')

class Deck:
    # ranks and suits are class attributes because they
    # should be shared by all decks
    __ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    __suits = 'clubs diamonds hearts spades'.split()

    def __init__(self):
        self._cards = [Card(rank, suit)
            for suit in self.__class__.__suits
                for rank in self.__class__.__ranks]

In [36]:
d = Deck()

# We can create a deck of cards, but it turns out it's not iterable...

for card in d:
    print(card)

TypeError: 'Deck' object is not iterable

In [37]:
# ...unless we refer to _cards directly

for card in d._cards:
    print(card, end=' ')

Card(rank='2', suit='clubs') Card(rank='3', suit='clubs') Card(rank='4', suit='clubs') Card(rank='5', suit='clubs') Card(rank='6', suit='clubs') Card(rank='7', suit='clubs') Card(rank='8', suit='clubs') Card(rank='9', suit='clubs') Card(rank='10', suit='clubs') Card(rank='J', suit='clubs') Card(rank='Q', suit='clubs') Card(rank='K', suit='clubs') Card(rank='A', suit='clubs') Card(rank='2', suit='diamonds') Card(rank='3', suit='diamonds') Card(rank='4', suit='diamonds') Card(rank='5', suit='diamonds') Card(rank='6', suit='diamonds') Card(rank='7', suit='diamonds') Card(rank='8', suit='diamonds') Card(rank='9', suit='diamonds') Card(rank='10', suit='diamonds') Card(rank='J', suit='diamonds') Card(rank='Q', suit='diamonds') Card(rank='K', suit='diamonds') Card(rank='A', suit='diamonds') Card(rank='2', suit='hearts') Card(rank='3', suit='hearts') Card(rank='4', suit='hearts') Card(rank='5', suit='hearts') Card(rank='6', suit='hearts') Card(rank='7', suit='hearts') Card(rank='8', suit='hear

In [38]:
# we also cannot find the length of the deck
# ...at least not without referring to `_cards` directly
print(len(d._cards))
print(len(d))

52


TypeError: object of type 'Deck' has no len()

## Making our deck iterable
* the Python data model allows us to accomplish quite a bit, just by implementing the __`__len__()`__ and __`__getitem__()`__ methods

In [39]:
# a deck of cards, round two
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class Deck(object):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'clubs diamonds hearts spades'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]
        #return list.__getitem__(self._cards, position)

In [40]:
deck = Deck()
len(deck)

52

In [41]:
for card in deck:
    print(card, end=' ')

Card(rank='2', suit='clubs') Card(rank='3', suit='clubs') Card(rank='4', suit='clubs') Card(rank='5', suit='clubs') Card(rank='6', suit='clubs') Card(rank='7', suit='clubs') Card(rank='8', suit='clubs') Card(rank='9', suit='clubs') Card(rank='10', suit='clubs') Card(rank='J', suit='clubs') Card(rank='Q', suit='clubs') Card(rank='K', suit='clubs') Card(rank='A', suit='clubs') Card(rank='2', suit='diamonds') Card(rank='3', suit='diamonds') Card(rank='4', suit='diamonds') Card(rank='5', suit='diamonds') Card(rank='6', suit='diamonds') Card(rank='7', suit='diamonds') Card(rank='8', suit='diamonds') Card(rank='9', suit='diamonds') Card(rank='10', suit='diamonds') Card(rank='J', suit='diamonds') Card(rank='Q', suit='diamonds') Card(rank='K', suit='diamonds') Card(rank='A', suit='diamonds') Card(rank='2', suit='hearts') Card(rank='3', suit='hearts') Card(rank='4', suit='hearts') Card(rank='5', suit='hearts') Card(rank='6', suit='hearts') Card(rank='7', suit='hearts') Card(rank='8', suit='hear

### ...but just by implementing __`__getitem__()`__, we get so much more!

In [42]:
# like indexing
deck[0], deck[-1]

(Card(rank='2', suit='clubs'), Card(rank='A', suit='spades'))

In [43]:
# ...and slicing!
deck[9:13]

[Card(rank='J', suit='clubs'),
 Card(rank='Q', suit='clubs'),
 Card(rank='K', suit='clubs'),
 Card(rank='A', suit='clubs')]

In [44]:
deck[12::13]

[Card(rank='A', suit='clubs'),
 Card(rank='A', suit='diamonds'),
 Card(rank='A', suit='hearts'),
 Card(rank='A', suit='spades')]

## What about a method to pick a random card?
* no need because Python already has a function to choose a random item from a sequence

In [45]:
import random
random.choice(deck)

Card(rank='K', suit='diamonds')

## Two big advantages of using special methods to leverage the Python data model
*  users of your classes don’t have to memorize arbitrary method names for standard operations (“How to get the number of items? Is it __`.size()`__, __`.length()`__, or what?”)
* it’s easier to benefit from the rich Python standard library and avoid reinventing the wheel, e.g., __`random.choice()`__